# TASK 7

In [10]:
import psycopg2 as psy
from psycopg2.extras import execute_values
import pandas as pd

In [11]:
#connections Handshake
conn = psy.connect(host = 'database-1.cpzopo51qbvs.us-east-2.rds.amazonaws.com',user ='postgres',password='postgres',database='recipe')
mycursor = conn.cursor()
conn.set_session(autocommit = True)

In [12]:
#Data Frame View
data = pd.read_json("/home/adarsh/train.json")
data.head(2)

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."


In [13]:
#DataFrame Cuisine with UNIQUE ID as index
data_t = data[['cuisine']]
data_t.drop_duplicates(subset='cuisine',inplace=True)
data_t = data_t.reset_index(drop = True).reset_index()


/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
data_s = data[['id','cuisine']]

#DataFrame for Table 1 
Table_1 = data_s.replace(to_replace=list(data_t['cuisine']),value =list(data_t['index'])) 
#replacing STRING Cuisine Name with UNIQUE Index


Table_1 =Table_1.reset_index()


,index,id,cuisine
0,0,10259,0
1,1,25693,1
2,2,20130,2
3,3,22213,3
4,4,13162,3
...,...,...,...
39769,39769,29109,16
39770,39770,11462,6
39771,39771,2238,16
39772,39772,41882,8


# TABLE 1

In [16]:
#Create__Table_1                        
query1 ="CREATE TABLE dishes (_ID INT,dish_id INT,cuisine_id INT,PRIMARY KEY (_ID))"
mycursor.execute(query1)

In [17]:
#Execution for Table 1
#Creating list of Tuples for TABLE 1
subset =Table_1[['index','id','cuisine']]
tuples=list(zip(*map(subset.get,subset)))
execute_values(mycursor,
              "INSERT INTO dishes (_id,dish_id,cuisine_id) VALUES %s",
               tuples)

# Table 2:

In [18]:
#Create__Table _2

query2 ="CREATE TABLE Cuisine (cuisine_id INT,cuisine_name VARCHAR,FOREIGN KEY (cuisine_id) REFERENCES dishes(_id))"
mycursor.execute(query2)

In [20]:
#Execution Table data for TABLE 2
Table_2_list = list(zip(data_t['index'],data_t['cuisine'])) #list of Tuples of Required Data For Table 2
try:
    execute_values(mycursor,
              "INSERT INTO Cuisine (cuisine_id,cuisine_name) VALUES %s",
              Table_2_list)
except ForeignKeyViolation:
    pass

# Table3

In [21]:
#Create__Table
query3 ="CREATE TABLE Ingredient (ingred_id INT PRIMARY KEY,ingred_name VARCHAR)"
mycursor.execute(query3)

In [22]:
#EXploding List in Ingredients For Table 3
t3 = data.explode('ingredients')
t3.drop_duplicates(subset='ingredients',inplace=True)
data_u =pd.DataFrame(t3['ingredients']).reset_index(drop=True).reset_index()


In [23]:

Table_3_list = list(zip(data_u['index'],data_u['ingredients']))  #list data Required for execution

In [24]:
#execution
execute_values(mycursor,
              "INSERT INTO Ingredient (ingred_id,ingred_name) VALUES %s",
               Table_3_list)

# Table 4:

In [32]:
#Create__Table_4
query3 ="CREATE TABLE Recipe (r_id INT , dish_id INT, ingred_id INT)"
mycursor.execute(query3)

In [25]:
#list required for replacing through dataFrame 
cuisine = list(data_t['cuisine'])
cuisine_id =list(data_t['index'])
ingredients=list(data_u['ingredients'])
ing_id =list(data_u['index'])


In [26]:
#replacing data values of Cuisine columns with Unique ID assigned 
Table_4 =data.explode('ingredients').replace(to_replace=cuisine,value =cuisine_id)

This will take 2 minutes

In [27]:
#replacing data values of ingredients with unique id assigned
sub_Table = Table_4['ingredients'].replace(to_replace=ingredients,value =ing_id)

In [28]:
#creating frames And concate
frame=[Table_4[['id','cuisine']],sub_Table]
result = pd.concat(frame,axis=1)

In [29]:
result.reset_index(inplace=True)
result

,index,id,cuisine,ingredients
0,0,10259,0,0
1,0,10259,0,1
2,0,10259,0,2
3,0,10259,0,3
4,0,10259,0,4
...,...,...,...,...
428270,39773,2362,7,3
428271,39773,2362,7,229
428272,39773,2362,7,157
428273,39773,2362,7,213


In [30]:
#Creating list of Tuples for TABLE 4
subset =result[['index','id','ingredients']]
tuples=list(zip(*map(subset.get,subset)))

In [ ]:
#execute
execute_values(mycursor,
              "INSERT INTO Recipe (r_id, dish_id, ingred_id) VALUES %s",
              tuples)